In [50]:
from jetbot import Camera, bgr8_to_jpeg
import traitlets
import ipywidgets as widgets
from IPython.display import display
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import time
from jetbot import Robot


In [51]:
camera = Camera.instance()
camera.stop()
camera = Camera.instance()
robot = Robot()
# 創建 Image 小工具，設置寬度和高度
image_widget = widgets.Image(format='jpeg', width=600, height=600)

binary_widget = widgets.Image(format='jpeg', width=600, height=600)

binary_ball_widget = widgets.Image(format='jpeg', width=600, height=600)



# 設定 ROI 的位置和大小
roi_x, roi_y, roi_w, roi_h = 100, 100, 10, 10 

mean_label = widgets.Label()


In [52]:
# Default 值都設定好了
r_lower = widgets.IntSlider(min=0, max=255, value=165, description='R Lower')
r_upper = widgets.IntSlider(min=0, max=255, value=255, description='R Upper')
g_lower = widgets.IntSlider(min=0, max=255, value=160, description='G Lower')
g_upper = widgets.IntSlider(min=0, max=255, value=255, description='G Upper')
b_lower = widgets.IntSlider(min=0, max=255, value=0, description='B Lower')
b_upper = widgets.IntSlider(min=0, max=255, value=150, description='B Upper')

r_lower_ball = widgets.IntSlider(min=0, max=255, value=160, description='R Lower (ball)')
r_upper_ball = widgets.IntSlider(min=0, max=255, value=255, description='R Upper (ball)')
g_lower_ball = widgets.IntSlider(min=0, max=255, value=50, description='G Lower (ball)')
g_upper_ball = widgets.IntSlider(min=0, max=255, value=130, description='G Upper (ball)')
b_lower_ball = widgets.IntSlider(min=0, max=255, value=0, description='B Lower (ball)')
b_upper_ball = widgets.IntSlider(min=0, max=255, value=130, description='B Upper (ball)')

# 創建最小和最大面積閾值的滑桿
min_area_slider = widgets.IntSlider(value=500, min=0, max=10000, step=50, description='最小面積')
max_area_slider = widgets.IntSlider(value=5000, min=0, max=20000, step=50, description='最大面積')
tracking_toggle = widgets.ToggleButton(value=False, description='啟用尋跡', button_style='success')



In [53]:
def sign(input):
    return input / abs(input) if input != 0 else 0  # 避免除以 0

def maxNum(input):
    if abs(input) > 100:
        return 100 * sign(input)
    else:
        return input

def SpeedControl(s_in, d_in):
    L = R = 0
    S = maxNum(s_in) 
    D = maxNum(d_in)/2
    

    L = S + D
    R = S - D

    L = maxNum(L)
    R = maxNum(R)
    L = L / 100
    R = R / 100

    robot.set_motors(L, R)
    return (L, R)

In [54]:
frame_count = 0
process_every_n_frames = 4  # 每處理 1 幀影像
process_ball_every_n_frames = 9  # 每處理 1 幀影像
track_ball = True
s_in = 22 # 固定前進速度
tick = 0

def update_images(change):
    global frame_count, flag, tick
    frame_count += 1
    if frame_count % process_ball_every_n_frames == 0:
        track_ball = True
    elif frame_count % process_every_n_frames == 0:
        track_ball = False
    else:
        return
    if tick > 0:
#         print(area)
        tick -= 1
        return
        
    frame = change['new']
    
    # 將影像轉換為 BGR 格式
    bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # 計算 ROI 的平均 RGB 值
    height, width, _ = bgr_frame.shape
    
    roi_frame = frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]
    mean_rgb = roi_frame.mean(axis=(0, 1))
    r_mean = mean_rgb[2]
    g_mean = mean_rgb[1]
    b_mean = mean_rgb[0]
    mean_label.value = f"ROI 平均 RGB 值：R={r_mean:.2f}, G={g_mean:.2f}, B={b_mean:.2f}"

    if track_ball:
        # 篩選紅色區域
        lower_rgb = np.array([b_lower_ball.value, g_lower_ball.value, r_lower_ball.value])
        upper_rgb = np.array([b_upper_ball.value, g_upper_ball.value, r_upper_ball.value])
    else:
        # 篩選黃色區域
        lower_rgb = np.array([b_lower.value, g_lower.value, r_lower.value])
        upper_rgb = np.array([b_upper.value, g_upper.value, r_upper.value])
        
    mask = cv2.inRange(frame, lower_rgb, upper_rgb)
    
    # 找出所有輪廓
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 最小面積過濾
    min_area = min_area_slider.value
    closest_centroid = None
    closest_distance = float('inf')
    frame_center_x = frame.shape[1] // 2  # 影像中間的 x 坐標
    
    if contours:
        largest_cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_cnt)
        M = cv2.moments(largest_cnt)
        
        if track_ball and tracking_toggle.value:
#         if track_ball:
#             tick = 3
            
            if area > 400:
                tick = 5
                print(area)
                print("hi")
#                 robot.set_motors(0.15, 0.2)
#                 time.sleep(1)
                # SpeedControl(-1 * s_in, 0)
                # time.sleep(0.4) # 

                SpeedControl(1.3, 25)
                time.sleep(0.6) # 
                SpeedControl(1 * s_in, 0)
                time.sleep(0.8) # 
                SpeedControl(1.3, -25)
                time.sleep(0.6) # 
                SpeedControl(1 * s_in, 0)
                time.sleep(0.5) # 
                SpeedControl(1.3, -20)
                time.sleep(0.5) # 
                SpeedControl(1 * s_in, 0)
                time.sleep(0.3) # 
                SpeedControl(0, 0)

        


#                 pass
        elif M['m00'] != 0:
            base_speed = 25
            turn_penalty = 0.3
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            closest_centroid = (cx, cy)
            error = cx - frame_center_x
            d_in = (error / frame.shape[1]) * 70  # 標準化誤差作為旋轉速度
            if tracking_toggle.value:
                SpeedControl(base_speed - (abs(d_in)* turn_penalty), d_in)

        # 在影像上畫出黃色區域中心點
        bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        if closest_centroid:
            cv2.circle(bgr_frame, closest_centroid, 5, (0, 255, 0), -1)
            cv2.putText(bgr_frame, f"({closest_centroid[0]}, {closest_centroid[1]})", 
                    (closest_centroid[0] + 10, closest_centroid[1] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    
    # 顯示原始影像與二值化影像
    cv2.rectangle(bgr_frame, (roi_x, roi_y), (roi_x + roi_w, roi_y + roi_h), (0, 0, 255), 2)
    _, frame_encoded = cv2.imencode('.jpg', cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB))
    image_widget.value = frame_encoded.tobytes()
    
    
    mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    _, binary_encoded = cv2.imencode('.jpg', mask_rgb)
    if track_ball:
        binary_ball_widget.value = binary_encoded.tobytes()
    else:
        binary_widget.value = binary_encoded.tobytes()
        



In [55]:
camera.observe(update_images, names='value')

In [56]:
display(widgets.HBox([image_widget,binary_widget,binary_ball_widget]),mean_label)


Label(value='')

In [58]:
# 顯示控制項
# display(widgets.VBox([b_lower, b_upper, g_lower, g_upper, r_lower, r_upper, min_area_slider, tracking_toggle]))
display(widgets.VBox([min_area_slider, tracking_toggle]))


In [59]:
camera.stop()
robot.stop()